In [15]:
# import requests
# import datetime

# def download_grib_file(min_lat, max_lat, min_lon, max_lon, forecast_hour, output_file):
#     """
#     Downloads a GRIB2 file from NOAA for a specified area and forecast hour.

#     Parameters:
#         min_lat (float): Minimum latitude of the area.
#         max_lat (float): Maximum latitude of the area.
#         min_lon (float): Minimum longitude of the area.
#         max_lon (float): Maximum longitude of the area.
#         forecast_hour (int): Forecast hour (e.g., 240 for 10-day forecast).
#         output_file (str): Path to save the downloaded GRIB2 file.
#     """
#     # Get the current UTC time
#     now = datetime.datetime.now(datetime.timezone.utc)
#     run_date = now.strftime("%Y%m%d")
#     run_hour = "00"  # Default to 00 UTC; change as needed

#     # Construct the URL for the GRIB2 file
#     file_name = f"gfs.t{run_hour}z.pgrb2b.0p25.f{forecast_hour:03d}"
#     base_url = f"https://nomads.ncep.noaa.gov/pub/data/nccf/com/gfs/prod/"
#     file_url = f"{base_url}gfs.{run_date}/{run_hour}/atmos/{file_name}"

#     try:
#         # Request to download the GRIB2 file
#         response = requests.get(file_url, stream=True)
#         response.raise_for_status()  # Check for HTTP errors

#         # Save the file locally
#         with open(output_file, "wb") as file:
#             for chunk in response.iter_content(chunk_size=8192):
#                 file.write(chunk)

#         print(f"GRIB2 file downloaded successfully: {output_file}")
#     except requests.exceptions.RequestException as e:
#         print(f"Error downloading GRIB2 file: {e}")
#         print(f"URL attempted: {file_url}")


In [62]:
import datetime

def get_latest_grib_time():
    """
    Calculates the latest GRIB file base time in the format required for downloading.

    Returns:
        str: The latest base time in 'YYYYMMDD/HH' format.
    """
    # Get the current UTC time
    now = datetime.datetime.now(datetime.UTC)
    print(f"Current UTC time: {now.strftime('%c')}")

    # GFS model updates every 6 hours (00, 06, 12, 18 UTC)
    # Find the most recent model run hour
    recent_run_hour = (now.hour // 6) * 6
    recent_run_time = now.replace(hour=recent_run_hour, minute=0, second=0, microsecond=0)

    # If the current time is before the most recent run's availability, go back one cycle
    # GFS files are typically available ~4 hours after the run starts
    if now < recent_run_time + datetime.timedelta(hours=4):
        recent_run_time -= datetime.timedelta(hours=6)

    # Format as 'YYYYMMDD/HH'
    return recent_run_time.strftime("%Y%m%d"), recent_run_time.strftime("%H")

# Example usage
latest_grib_time = get_latest_grib_time()
print(f"Latest GRIB time: {latest_grib_time}")

Current UTC time: Wed Dec  4 16:44:36 2024
Latest GRIB time: ('20241204', '12')


In [80]:
# Example usage
##################
# Define the geographic bounds (Note: bounds are ignored for GRIB2b files)
bounds = {'lat': (32.0, 34.5),
          'lng': (-122.0, -116.0) }
##################
# Waypoints for the Channel islands 500
waypoints = [(34.40161, -119.69426),  #SBYC (start)
             (34.08265, -119.98985),  #Santa Cruz channel (north)
             (33.95741, -119.91079),  #Santa Cruz channel (south)
             (32.96459, -121.34031),  #Western Mark
             (31.64010, -119.82025),  #Southern Mark
             (32.75008, -117.49959),  #Finish
            ]
##################
# find the time of the latest forecast calculation
(FCdate, FCtime) = get_latest_grib_time()
print(FCdate, FCtime)
##################
# at t=0, find the wind a the first waypoint
simulation_time=120
grib_file = download_nomads_gfs_forecast_file(FCdate, FCtime, bounds['lat'], bounds['lng'], simulation_time)
get_wind_at_location(grib_file, waypoints[0][0], waypoints[0][1], simulation_time)


Current UTC time: Wed Dec  4 23:45:26 2024
20241204 18
238.0 244.0
https://nomads.ncep.noaa.gov/cgi-bin/filter_gfs_0p25_1hr.pl?dir=%2Fgfs.20241204%2F18%2Fatmos&file=gfs.t18z.pgrb2.0p25.f120&var_UGRD=on&var_VGRD=on&lev_10_m_above_ground=on&subregion=&toplat=34.5&leftlon=238.0&rightlon=244.0&bottomlat=32.0
gfs.20241204-18-gfs.t18z.pgrb2.0p25.f120
starting download
GRIB2 file downloaded successfully: gfs.20241204-18-gfs.t18z.pgrb2.0p25.f120


In [78]:
# docs here: https://nomads.ncep.noaa.gov/gribfilter.php?ds=gfs_0p25_1hr

def download_nomads_gfs_forecast_file(grib_date, grib_time, lat_bounds, lng_bounds, simulation_time):
    min_lng = lng_bounds[0] if lng_bounds[0]>0 else lng_bounds[0]+360
    max_lng = lng_bounds[1] if lng_bounds[1]>0 else lng_bounds[1]+360
    print(min_lng,max_lng)
    min_lat = lat_bounds[0]
    max_lat = lat_bounds[1]
    #https://nomads.ncep.noaa.gov/cgi-bin/filter_gfs_0p25_1hr.pl?dir=%2Fgfs.20241203%2F12%2Fatmos&file=gfs.t12z.pgrb2.0p25.f240&var_UGRD=on&var_VGRD=on&lev_10_m_above_ground=on&subregion=&toplat=34.5&leftlon=238&rightlon=244&bottomlat=32
    url = f"https://nomads.ncep.noaa.gov/cgi-bin/filter_gfs_0p25_1hr.pl?dir=%2Fgfs.{grib_date}%2F{grib_time}%2Fatmos&file=gfs.t{grib_time}z.pgrb2.0p25.f{simulation_time:03}&var_UGRD=on&var_VGRD=on&lev_10_m_above_ground=on&subregion=&toplat={max_lat}&leftlon={min_lng}&rightlon={max_lng}&bottomlat={min_lat}"
    print(url)
    output_file = f"gfs.{grib_date}-{grib_time}-gfs.t{grib_time}z.pgrb2.0p25.f{simulation_time:03}"
    print(output_file)
    
    try:
        # Make the request to download the GRIB2 file
        print(f"starting download")
        response = requests.get(url, stream=True)
        response.raise_for_status()  # Check for HTTP errors
        
        # Save the file locally
        with open(output_file, "wb") as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)
        
        print(f"GRIB2 file downloaded successfully: {output_file}")
        return output_file
    except requests.exceptions.RequestException as e:
        print(f"Error downloading GRIB2 file: {e}")
        raise e
    

In [29]:
import pygrib
import numpy as np
from math import atan2, degrees, sqrt

def get_wind_at_location(grib_file, lat, lon, forecast_time):
    """
    Extracts wind speed and direction at a given GPS coordinate and time from a GRIB2 file.

    Parameters:
        grib_file (str): Path to the GRIB2 file.
        lat (float): Latitude of the location.
        lon (float): Longitude of the location.
        forecast_time (int): Forecast hour to extract data for.

    Returns:
        dict: A dictionary with wind speed (m/s) and direction (degrees).
    """
    try:
        # Open the GRIB2 file
        with pygrib.open(grib_file) as grbs:
            # Filter messages for U and V wind components at 10m above ground and the given forecast time
            u_msg = grbs.select(name="U component of wind", level=100, forecastTime=forecast_time)[0]
            v_msg = grbs.select(name="V component of wind", level=1000, forecastTime=forecast_time)[0]

            # Extract the data and corresponding lat/lon grid
            u_data, lats, lons = u_msg.data(lat1=lat, lat2=lat, lon1=lon, lon2=lon)
            v_data, _, _ = v_msg.data(lat1=lat, lat2=lat, lon1=lon, lon2=lon)

            # Interpolate U and V components at the exact point
            u_value = u_data[0][0]
            v_value = v_data[0][0]

            # Calculate wind speed (m/s) and direction (degrees)
            wind_speed = sqrt(u_value**2 + v_value**2)
            wind_direction = (270 - degrees(atan2(v_value, u_value))) % 360

            return {"wind_speed": wind_speed, "wind_direction": wind_direction}

    except Exception as e:
        print(f"Error processing GRIB2 file: {e}")
        return None




In [81]:
!ls

GetWeatherData.ipynb
README.md
Test.ipynb
Untitled.ipynb
first_figure.html
gfs.20241204-18-gfs.t18z.pgrb2.0p25.f000
gfs.20241204-18-gfs.t18z.pgrb2.0p25.f120
gfs.t12z.pgrb2.0p25.f240
gfs_10day_forecast.grib2
install.txt
run_docker.sh
test_data
venv


In [82]:
gps_lat = 33.0  # Example latitude
gps_lon = -118.0 + 360  # Example longitude
get_wind_at_location("gfs.20241204-18-gfs.t18z.pgrb2.0p25.f000", gps_lat, gps_lon, 240)

Error processing GRIB2 file: no matches found


In [85]:
gps_lat = 33.0  # Example latitude
gps_lon = -118.0 + 360  # Example longitude
grbs = pygrib.open("gfs.20241204-18-gfs.t18z.pgrb2.0p25.f000")
for grb in grbs:
    print(grb)  # Prints metadata about each message in the file
    print(f"Forecast time: {grb.forecastTime} hours, Parameter: {grb.name}")
u_msg = grbs.select(name="10 metre U wind component", level=10, forecastTime=0)[0]
print(u_msg)
print(u_msg.data())
x = u_msg.data()

1:10 metre U wind component:m s**-1 (instant):regular_ll:heightAboveGround:level 10 m:fcst time 0 hrs:from 202412041800
Forecast time: 0 hours, Parameter: 10 metre U wind component
2:10 metre V wind component:m s**-1 (instant):regular_ll:heightAboveGround:level 10 m:fcst time 0 hrs:from 202412041800
Forecast time: 0 hours, Parameter: 10 metre V wind component
1:10 metre U wind component:m s**-1 (instant):regular_ll:heightAboveGround:level 10 m:fcst time 0 hrs:from 202412041800
(array([[ 4.88861572e+00,  4.87861572e+00,  4.84861572e+00,
         4.69861572e+00,  4.42861572e+00,  4.12861572e+00,
         3.91861572e+00,  3.71861572e+00,  3.46861572e+00,
         3.12861572e+00,  2.87861572e+00,  2.69861572e+00,
         2.63861572e+00,  2.65861572e+00,  2.77861572e+00,
         2.75861572e+00,  2.48861572e+00,  2.13861572e+00,
         1.69861572e+00,  1.58861572e+00,  2.22861572e+00,
         1.76861572e+00,  1.35861572e+00,  1.38861572e+00,
        -2.01384277e-01],
       [ 4.36861572

In [89]:
lats, lngs = grb.latlons()
print(lats)
print()
print(lngs)

[[32.   32.   32.   32.   32.   32.   32.   32.   32.   32.   32.   32.
  32.   32.   32.   32.   32.   32.   32.   32.   32.   32.   32.   32.
  32.  ]
 [32.25 32.25 32.25 32.25 32.25 32.25 32.25 32.25 32.25 32.25 32.25 32.25
  32.25 32.25 32.25 32.25 32.25 32.25 32.25 32.25 32.25 32.25 32.25 32.25
  32.25]
 [32.5  32.5  32.5  32.5  32.5  32.5  32.5  32.5  32.5  32.5  32.5  32.5
  32.5  32.5  32.5  32.5  32.5  32.5  32.5  32.5  32.5  32.5  32.5  32.5
  32.5 ]
 [32.75 32.75 32.75 32.75 32.75 32.75 32.75 32.75 32.75 32.75 32.75 32.75
  32.75 32.75 32.75 32.75 32.75 32.75 32.75 32.75 32.75 32.75 32.75 32.75
  32.75]
 [33.   33.   33.   33.   33.   33.   33.   33.   33.   33.   33.   33.
  33.   33.   33.   33.   33.   33.   33.   33.   33.   33.   33.   33.
  33.  ]
 [33.25 33.25 33.25 33.25 33.25 33.25 33.25 33.25 33.25 33.25 33.25 33.25
  33.25 33.25 33.25 33.25 33.25 33.25 33.25 33.25 33.25 33.25 33.25 33.25
  33.25]
 [33.5  33.5  33.5  33.5  33.5  33.5  33.5  33.5  33.5  33.5  33.5  

In [87]:
grb.values.shape

(11, 25)

In [53]:
x[0].shape

(11, 25)

In [52]:
x[1].shape

(11, 25)

In [37]:
grbs.select(name="U component of wind", level=10, forecastTime=0)

ValueError: no matches found

In [30]:
# Example usage
# Path to the GRIB2 file
grib_file_path = "gfs_10day_forecast.grib2"

# Specify the GPS coordinates and forecast time
gps_lat = 33.0  # Example latitude
gps_lon = -120.0  # Example longitude
forecast_hour = 0  

# Get wind speed and direction
result = get_wind_at_location(grib_file_path, gps_lat, gps_lon, forecast_hour)
if result:
    print(f"Wind speed: {result['wind_speed']} m/s")
    print(f"Wind direction: {result['wind_direction']} degrees")

Error processing GRIB2 file: no matches found


In [28]:
grib_file_path = "gfs_10day_forecast.grib2"
with pygrib.open(grib_file_path) as grbs:
    for grb in grbs:
        print(grb)  # Prints metadata about each message in the file

1:Geopotential height:gpm (instant):regular_ll:isobaricInhPa:level 100 Pa:fcst time 240 hrs:from 202412030000
2:Temperature:K (instant):regular_ll:isobaricInhPa:level 100 Pa:fcst time 240 hrs:from 202412030000
3:Relative humidity:% (instant):regular_ll:isobaricInhPa:level 100 Pa:fcst time 240 hrs:from 202412030000
4:U component of wind:m s**-1 (instant):regular_ll:isobaricInhPa:level 100 Pa:fcst time 240 hrs:from 202412030000
5:V component of wind:m s**-1 (instant):regular_ll:isobaricInhPa:level 100 Pa:fcst time 240 hrs:from 202412030000
6:Absolute vorticity:s**-1 (instant):regular_ll:isobaricInhPa:level 100 Pa:fcst time 240 hrs:from 202412030000
7:Ozone mixing ratio:kg kg**-1 (instant):regular_ll:isobaricInhPa:level 100 Pa:fcst time 240 hrs:from 202412030000
8:Geopotential height:gpm (instant):regular_ll:isobaricInhPa:level 200 Pa:fcst time 240 hrs:from 202412030000
9:Temperature:K (instant):regular_ll:isobaricInhPa:level 200 Pa:fcst time 240 hrs:from 202412030000
10:Relative humidity